<h2> In this notebook. we will do stresstesting of our VaR montel Carlo 95% and how robust it is under historical stress and hypothetical stress

<h5> While VaR answers "How bad can losses be on a normal bad day?", Stress testing answers "What happens if the world is NOT normal?".

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [3]:
returns = pd.read_csv("../data/portfolio_log_returns.csv", index_col=0, parse_dates=True)
print(f"\nPortfolio returns head:\n{returns.head()}")

VaR_series = pd.read_csv("../data/var_mc_95.csv", index_col=0, parse_dates=True)
print(f"\nVaR series head:\n{VaR_series.head()}")

data = pd.concat([returns, VaR_series], axis=1, join='inner').dropna()
data.head()


Portfolio returns head:
            port_log
DATE                
2016-01-26  0.008944
2016-01-27 -0.005780
2016-01-28  0.006425
2016-01-29  0.010268
2016-02-01  0.001797

VaR series head:
            VaR_MC_95
DATE                 
2017-01-26  -0.007791
2017-01-27  -0.007850
2017-01-30  -0.007807
2017-01-31  -0.007859
2017-02-01  -0.007840


,port_log,VaR_MC_95
DATE,,
2017-01-26,-0.003169,-0.007791
2017-01-27,0.000229,-0.007850
2017-01-30,-0.003953,-0.007807
2017-01-31,0.003675,-0.007859
2017-02-01,-0.001157,-0.007840


<h2> Historical stress testing

In [4]:
# 1. Define COVID stress window
covid_start = '2020-03-01'
covid_end = '2020-06-30'
covid_window = data.loc[covid_start:covid_end]

covid_window.head()

,port_log,VaR_MC_95
DATE,,
2020-03-02,0.032890,-0.008269
2020-03-03,-0.016712,-0.008877
2020-03-04,0.023188,-0.009075
2020-03-05,-0.018127,-0.009261
2020-03-06,-0.006164,-0.009489


In [7]:
# 2. Compute stressed losses during the COVID period
covid_losses = covid_window['port_log']

# 3. Compare realized losses vs VaR
covid_violations = covid_window['port_log'] < covid_window['VaR_MC_95'] # higher losses (more negative) than VaR indicate breaches

print("COVID stress period summary:")
print(f"Number of days: {len(covid_losses)}")
print(f"VaR breaches: {covid_violations.sum()}")
print(f"Max daily loss: {covid_losses.min():.4f}")
print(f"Average daily loss: {covid_losses.mean():.4f}")

COVID stress period summary:
Number of days: 85
VaR breaches: 11
Max daily loss: -0.0740
Average daily loss: 0.0004


<h5> During the COVID stress period, realized losses exceeded the model's 95% VaR on 12.9% of days (11 over 85 days). This indicates that the VaR model significantly underestimated tail risks during crisis regimes.

<h2> Hypothetical Stress Testing

In [8]:
# 1. Reestimate base parameters
mu = returns['port_log'].mean()
sigma = returns['port_log'].std()

# 2. Apply stress scenario: 2x volatility shock
stress_multiplier = 2.0  # 2x volatility shock
sigma_stressed = sigma * stress_multiplier

# 3. Recalculate VaR under stressed parameters
n_sim = 100_000
simulated_stress_returns = np.random.normal(mu, sigma_stressed, n_sim)

# 4. Calculate stressed VaR at 95% confidence level
VaR_stress_95 = np.percentile(simulated_stress_returns, 5)
ES_stress_95 = simulated_stress_returns[simulated_stress_returns <= VaR_stress_95].mean()

print(f"Stressed VaR (95%): {VaR_stress_95:.4f}")
print(f"Stressed ES (95%): {ES_stress_95:.4f}")


Stressed VaR (95%): -0.0236
Stressed ES (95%): -0.0296


<h5>

- Monte Carlo Simulated VaR for Portfolio at 95% confidence: -0.0116
- Monte Carlo Expected Shortfall for Portfolio at 95% confidence: -0.0147

- Stressed VaR (95%): -0.0236
- Stressed ES (95%): -0.0296

Under a hypothetical doubling of market volatility, the portfolio's 95% VaR increases from -0.0116 to -0.0236, highlighting significant sensitivity to volatility assumptions and model risk under stressed conditions.